In [0]:
# Load the dataset from DBFS (Databricks Filesystem)
file_path = "/databricks-datasets/samples/population-vs-price/data_geo.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- 2014 rank: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- 2014 Population estimate: integer (nullable = true)
 |-- 2015 median sales price: double (nullable = true)

+---------+-------------+-------+----------+------------------------+-----------------------+
|2014 rank|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+-------+----------+------------------------+-----------------------+
|      101|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|      125|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|      122|       Mobile|Alabama|        AL|                  194675|                  122.5|
|      114|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|       64|Anchorage[19]| Alaska|        AK|                  30101

In [0]:
# Select relevant columns for analysis
df_selected = df.select(
    "City", "State", "State Code", 
    "2014 Population estimate", "2015 median sales price"
)
df_selected.show(5)

+-------------+-------+----------+------------------------+-----------------------+
|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+-------------+-------+----------+------------------------+-----------------------+
|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|       Mobile|Alabama|        AL|                  194675|                  122.5|
|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|Anchorage[19]| Alaska|        AK|                  301010|                   null|
+-------------+-------+----------+------------------------+-----------------------+
only showing top 5 rows



In [0]:
df_selected = df.select("City","2014 Population estimate").orderBy("2014 Population estimate", ascending=False).limit(10)  
df_selected.display()

City,2014 Population estimate
New York[6],8491079
Los Angeles,3928864
Chicago,2722389
Houston[7],2239558
Philadelphia[8],1560297
Phoenix,1537058
San Antonio,1436697
San Diego,1381069
Dallas,1281047
San Jose,1015785


In [0]:
from pyspark.sql import SparkSession  
from pyspark.sql.functions import avg  

df_avg_sales_price = df.groupBy("State").agg(avg("2015 median sales price").alias("Average Median Sales Price"))  
df_avg_sales_price.show()  

+--------------------+--------------------------+
|               State|Average Median Sales Price|
+--------------------+--------------------------+
|                Utah|                     243.3|
|             Hawai'i|                     699.3|
|           Minnesota|                     209.4|
|                Ohio|        112.86666666666667|
|            Arkansas|                     131.8|
|              Oregon|        231.53333333333333|
|               Texas|         173.4777777777778|
|        North Dakota|                     180.0|
|        Pennsylvania|                     183.8|
|         Connecticut|        250.13333333333333|
|            Nebraska|        149.14999999999998|
|              Nevada|                     237.3|
|          Washington|        266.29999999999995|
|            Illinois|        122.87499999999999|
|            Oklahoma|                     145.1|
|District of Columbia|                     367.8|
|              Alaska|                      null|


In [0]:
df_selected.filter(df_selected["2014 Population estimate"] > 100000).show()  

+---------------+------------------------+
|           City|2014 Population estimate|
+---------------+------------------------+
|    New York[6]|                 8491079|
|    Los Angeles|                 3928864|
|        Chicago|                 2722389|
|     Houston[7]|                 2239558|
|Philadelphia[8]|                 1560297|
|        Phoenix|                 1537058|
|    San Antonio|                 1436697|
|      San Diego|                 1381069|
|         Dallas|                 1281047|
|       San Jose|                 1015785|
+---------------+------------------------+



In [0]:
df.filter((df["2015 median sales price"].isNull()) | (df["2015 median sales price"] == 0)).show()  

+---------+-------------+----------+----------+------------------------+-----------------------+
|2014 rank|         City|     State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+----------+----------+------------------------+-----------------------+
|       64|Anchorage[19]|    Alaska|        AK|                  301010|                   null|
|       78|     Chandler|   Arizona|        AZ|                  254276|                   null|
|       86|  Gilbert[20]|   Arizona|        AZ|                  239277|                   null|
|       88|     Glendale|   Arizona|        AZ|                  237517|                   null|
|       38|         Mesa|   Arizona|        AZ|                  464704|                   null|
|      148|       Peoria|   Arizona|        AZ|                  166934|                   null|
|       95|   Scottsdale|   Arizona|        AZ|                  230512|                   null|
|      215|     Surprise|   Ar